# <b><span style='color:#F1A424'>AutoML - Regression - Advertisment Sales Prediction </span> </b>

### Disclaimer
Please note, the Vantage Functions via SQLAlchemy feature is a preview/beta code release with limited functionality (the “Code”). As such, you acknowledge that the Code is experimental in nature and that the Code is provided “AS IS” and may not be functional on any machine or in any environment. TERADATA DISCLAIMS ALL WARRANTIES RELATING TO THE CODE, EXPRESS OR IMPLIED, INCLUDING, BUT NOT LIMITED TO, ANY WARRANTIES AGAINST INFRINGEMENT OF THIRD-PARTY RIGHTS, MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE.

TERADATA SHALL NOT BE RESPONSIBLE OR LIABLE WITH RESPECT TO ANY SUBJECT MATTER OF THE CODE UNDER ANY CONTRACT, NEGLIGENCE, STRICT LIABILITY OR OTHER THEORY 
    (A) FOR LOSS OR INACCURACY OF DATA OR COST OF PROCUREMENT OF SUBSTITUTE GOODS, SERVICES OR TECHNOLOGY, OR 
    (B) FOR ANY INDIRECT, INCIDENTAL OR CONSEQUENTIAL DAMAGES INCLUDING, BUT NOT LIMITED TO LOSS OF REVENUES AND LOSS OF PROFITS. TERADATA SHALL NOT BE RESPONSIBLE FOR ANY MATTER BEYOND ITS REASONABLE CONTROL.

Notwithstanding anything to the contrary: 
    (a) Teradata will have no obligation of any kind with respect to any Code-related comments, suggestions, design changes or improvements that you elect to provide to Teradata in either verbal or written form (collectively, “Feedback”), and 
    (b) Teradata and its affiliates are hereby free to use any ideas, concepts, know-how or techniques, in whole or in part, contained in Feedback: 
        (i) for any purpose whatsoever, including developing, manufacturing, and/or marketing products and/or services incorporating Feedback in whole or in part, and 
        (ii) without any restrictions or limitations, including requiring the payment of any license fees, royalties, or other consideration. 

## <b> Problem overview:</b>
    

**Dataset Used : Advertising Sales Dataset**

**Features**:

- `TV`: Advertising done on TV.
- `Radio`: Advertising done on Radio.
- `Newspapaer`: Advertising done on Newspaper.

**Target Variable**:

- `Sales`: The Sales received after advertisement.
    
**Objective**:

The primary objective is typically to build a model that can accurately predict sales received based on advertisement.

**Usecase**:

Here, we will use AutoML(Automated Machine Learning) functionality to automate the entire process of developing a predictive model. It will perform feature exploration, feature engineering, data preparation, model training and evaluation on dataset in auto run and at end we will get leaderboard containined different models along with their performance. Model will also have rank associated with them which indicates which is best performing model for given data followed by other models.

In [1]:
# Importing AutoML from teradataml
from teradataml import AutoML, AutoRegressor

In [2]:
# Importing other important libraries
import getpass
from teradataml import create_context, remove_context
from teradataml import DataFrame
from teradataml import load_example_data
from teradataml import TrainTestSplit

In [3]:
# Create the connection.
host = getpass.getpass("Host: ")
username = getpass.getpass("Username: ")
password = getpass.getpass("Password: ")

con = create_context(host=host, username=username, password=password)

Host:  ········
Username:  ········
Password:  ········


## <b><span style='color:#F1A424'>| 1.</span> Loading Deployed Models - 'Advertising_top_5_models' </b>

### <b><span style='color:#F1A424'>| 1.1.</span> Loading Model </b>

In [4]:
# Creating AutoML object

aml=AutoML()

In [5]:
# Loading models

models_1 = aml.load('Advertising_top_5_models')

In [6]:
# Display loaded models

models_1

RANK          MODEL_ID FEATURE_SELECTION       MAE       MSE      MSLE  \
0     1             KNN_1               rfe  1.055790  2.023639  0.009446   
1     2             GLM_1               rfe  1.229208  2.311243  0.011298   
2     3  DECISIONFOREST_1               rfe  1.529687  3.415041  0.017554   
3     4             GLM_2               pca  1.647833  4.197219  0.019330   
4     5         XGBOOST_2               pca  1.594226  4.208329  0.019006   

        MAPE       MPE      RMSE     RMSLE        ME        R2        EV  \
0   7.151976  2.392202  1.422547  0.097189  4.203217  0.922555  0.927300   
1   8.779032 -0.090825  1.520277  0.106292  3.139551  0.911548  0.912892   
2  11.224770 -2.792731  1.847983  0.132491  3.880000  0.869305  0.869330   
3  12.127324 -3.956709  2.048712  0.139032  4.823611  0.839371  0.839389   
4  10.414319  2.439263  2.051421  0.137864  5.575339  0.838945  0.846603   

        MPD       MGD                      DATA_TABLE  ADJUSTED_R2  
0  0.149030  0.011623  ml__sales_rfe_1723391384424642     0.921555  
1  0.164516  0.013240  ml__sales_rfe_1723391384424642     0.910406  
2  0.250371  0.020938  ml__sales_rfe_1723391384424642     0.867619  
3  0.281034  0.021870  ml__sales_pca_1723396755457502     0.836242  
4  0.293122  0.024563  ml__sales_pca_1723396755457502     0.835808

### <b><span style='color:#F1A424'>| 1.2.</span> Get Loaded Model Hyperparameters</b>

In [7]:
aml.model_hyperparameters(rank=1, use_loaded_models=True)

{'response_column': 'sales',
 'name': 'knn',
 'model_type': 'Regression',
 'k': 5,
 'id_column': 'id',
 'voting_weight': 1.0,
 'persist': False,
 'max_models': 1}

In [8]:
aml.model_hyperparameters(rank=5, use_loaded_models=True)

{'response_column': 'sales',
 'name': 'xgboost',
 'model_type': 'Regression',
 'column_sampling': 1,
 'min_impurity': 0.1,
 'lambda1': 0.1,
 'shrinkage_factor': 0.5,
 'max_depth': 7,
 'min_node_size': 2,
 'iter_num': 10,
 'seed': 42,
 'persist': False,
 'max_models': 1}

### <b><span style='color:#F1A424'>| 1.3.</span> Loading Dataset</b>

In [9]:
### Loading Dataset for Prediction

load_example_data('teradataml','advertising')
df = DataFrame('advertising')

In [10]:
# Display data

df

radio  newspaper  sales
TV                            
228.0   37.7       32.0   21.5
239.3   15.5       27.3   20.7
241.7   38.0       23.2   21.8
7.3     28.1       41.4    5.5
296.4   36.3      100.9   23.8
230.1   37.8       69.2   22.1
199.1   30.6       38.7   18.3
163.3   31.6       52.9   16.9
94.2     4.9        8.1   14.0
218.5    5.4       27.4   17.2

### <b><span style='color:#F1A424'>| 1.4.</span> Generating Prediction & Performance Metrics</b>

In [11]:
# Generate prediction using some data rows and model rank

prediction = aml.predict(df, rank=1)

Generating prediction using:
Model Name: KNN
Feature Selection: rfe


In [12]:
prediction

id  prediction  sales
0  22    7.599999    7.6
1  14   19.799999   19.8
2  19   16.800001   16.8
3  12   17.299999   17.3
4  41   20.899997   20.9
5  48    5.600002    5.6
6  28   17.100002   17.1
7  31   21.500000   21.5
8  10   16.996972   15.0
9  15   17.200000   17.2

In [13]:
# Generate performance metrics

performance_metric = aml.evaluate(df, rank=1)

Generating performance metrics using:
Model Name: KNN
Feature Selection: rfe


In [14]:
performance_metric


############ result Output ############

        MAE       MSE     MSLE      MAPE       MPE      RMSE     RMSLE        ME        R2        EV       MPD       MGD
0  0.154975  0.255818  0.00118  1.179964 -0.389643  0.505785  0.034353  4.345287  0.990791  0.990814  0.017475  0.001366


In [15]:
# Generate prediction using data and model rank

prediction = aml.predict(df, rank=4)

Generating prediction using:
Model Name: GLM
Feature Selection: pca


In [16]:
prediction

id  prediction  sales
0   40   12.935503   10.4
1   80   22.717113   26.2
2   99   18.689160   17.1
3   61   17.030077   19.6
4  221   12.687919   12.6
5   17    9.234658    5.3
6  162   12.319389   11.9
7  122    9.239742    6.9
8  101   21.667870   25.4
9  183   16.698419   18.0

In [17]:
# Generate performance metrics

performance_metric = aml.evaluate(df, rank=4)

Generating performance metrics using:
Model Name: GLM
Feature Selection: pca


In [18]:
performance_metric


############ result Output ############

       MAE       MSE      MSLE       MAPE       MPE      RMSE     RMSLE        ME        R2        EV       MPD       MGD
0  1.69903  4.566835  0.035048  16.376698 -9.841271  2.137015  0.187211  7.770491  0.835607  0.840585  0.366177  0.037724


## <b><span style='color:#F1A424'>| 2.</span> Loading Deployed Models - 'Advertising_mixed_models' </b>

### <b><span style='color:#F1A424'>| 2.1.</span> Loading Model </b>

In [19]:
# Loading models

models_2 = aml.load('Advertising_mixed_models')

In [20]:
models_2

RANK   MODEL_ID FEATURE_SELECTION       MAE       MSE      MSLE       MAPE  \
0     1      GLM_1               rfe  1.229208  2.311243  0.011298   8.779032   
1     2      GLM_2               pca  1.647833  4.197219  0.019330  12.127324   
2     3  XGBOOST_2               pca  1.594226  4.208329  0.019006  10.414319   

        MPE      RMSE     RMSLE        ME        R2        EV       MPD  \
0 -0.090825  1.520277  0.106292  3.139551  0.911548  0.912892  0.164516   
1 -3.956709  2.048712  0.139032  4.823611  0.839371  0.839389  0.281034   
2  2.439263  2.051421  0.137864  5.575339  0.838945  0.846603  0.293122   

        MGD                      DATA_TABLE  ADJUSTED_R2  
0  0.013240  ml__sales_rfe_1723397854656713     0.910406  
1  0.021870  ml__sales_pca_1723389958236912     0.836242  
2  0.024563  ml__sales_pca_1723389958236912     0.835808

### <b><span style='color:#F1A424'>| 2.2.</span> Generating Prediction & Performance Metrics</b>

In [21]:
# Generate prediction using data and model rank

prediction = aml.predict(df, rank=2)

Generating prediction using:
Model Name: GLM
Feature Selection: pca


In [22]:
prediction

id  prediction  sales
0   40   12.935503   10.4
1   80   22.717113   26.2
2   99   18.689160   17.1
3   61   17.030077   19.6
4  221   12.687919   12.6
5   17    9.234658    5.3
6  162   12.319389   11.9
7  122    9.239742    6.9
8  101   21.667870   25.4
9  183   16.698419   18.0

In [23]:
# Generate performance metrics

performance_metric = aml.evaluate(df, rank=2)

Generating performance metrics using:
Model Name: GLM
Feature Selection: pca


In [24]:
performance_metric


############ result Output ############

       MAE       MSE      MSLE       MAPE       MPE      RMSE     RMSLE        ME        R2        EV       MPD       MGD
0  1.69903  4.566835  0.035048  16.376698 -9.841271  2.137015  0.187211  7.770491  0.835607  0.840585  0.366177  0.037724


## <b><span style='color:#F1A424'>| 3.</span> Loading Deployed Models - 'Advertising_range_models' </b>

### <b><span style='color:#F1A424'>| 3.1.</span> Loading Model</b>

In [25]:
# Creating another AutoML object

obj=AutoML()

In [26]:
# Loading models

models_3 = obj.load('Advertising_range_models')

In [27]:
models_3

RANK          MODEL_ID FEATURE_SELECTION       MAE       MSE      MSLE  \
0     1             GLM_1               rfe  1.229208  2.311243  0.011298   
1     2  DECISIONFOREST_1               rfe  1.529687  3.415041  0.017554   
2     3             GLM_2               pca  1.647833  4.197219  0.019330   
3     4         XGBOOST_2               pca  1.594226  4.208329  0.019006   
4     5             SVM_2               pca  1.719796  4.371814  0.015838   
5     6  DECISIONFOREST_2               pca  1.728646  5.222329  0.020647   

        MAPE       MPE      RMSE     RMSLE        ME        R2        EV  \
0   8.779032 -0.090825  1.520277  0.106292  3.139551  0.911548  0.912892   
1  11.224770 -2.792731  1.847983  0.132491  3.880000  0.869305  0.869330   
2  12.127324 -3.956709  2.048712  0.139032  4.823611  0.839371  0.839389   
3  10.414319  2.439263  2.051421  0.137864  5.575339  0.838945  0.846603   
4  11.118059 -1.933912  2.090888  0.125850  5.371160  0.832689  0.842874   
5  11.687600  0.109286  2.285242  0.143692  5.575000  0.800139  0.812442   

        MPD       MGD                      DATA_TABLE  ADJUSTED_R2  
0  0.164516  0.013240  ml__sales_rfe_1723390582128520     0.910406  
1  0.250371  0.020938  ml__sales_rfe_1723390582128520     0.867619  
2  0.281034  0.021870  ml__sales_pca_1723392367146603     0.836242  
3  0.293122  0.024563  ml__sales_pca_1723392367146603     0.835808  
4  0.266629  0.018849  ml__sales_pca_1723392367146603     0.829430  
5  0.334104  0.024520  ml__sales_pca_1723392367146603     0.796246

### <b><span style='color:#F1A424'>| 3.2.</span> Generating Prediction & Performance Metrics</b>

In [28]:
# Generate prediction using data and model rank

prediction = obj.predict(df.iloc[:80], rank=1)

Generating prediction using:
Model Name: GLM
Feature Selection: rfe


In [29]:
prediction

id  prediction  sales
0   80    4.916385    5.3
1   40    9.409142    6.6
2  120    9.800749    8.0
3   17    8.930491   11.0
4  160    9.813268    9.2
5  154    9.658645   13.2
6  162   10.432900   13.7
7  202   16.100691   16.7
8  122   14.695747   15.3
9   19   12.493748   11.9

In [30]:
# Generate performance metrics

performance_metric = obj.evaluate(df.iloc[:80], rank=1)

Generating performance metrics using:
Model Name: GLM
Feature Selection: rfe


In [31]:
performance_metric


############ result Output ############

        MAE       MSE      MSLE       MAPE       MPE     RMSE    RMSLE        ME        R2       EV       MPD       MGD
0  1.137393  2.916206  0.045912  17.791905 -6.704899  1.70769  0.21427  7.507286  0.666776  0.66831  0.361321  0.051383


In [32]:
remove_context()

True